In [1]:
using JuMP

In [2]:
using CPLEX

In [3]:
model=Model(CPLEX.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: CPLEX

In [4]:
@variable(model, s[i=1:3,j=1:2],lower_bound=0) # decision variable: quantity supplied from ith grower to jth plant


3×2 Matrix{VariableRef}:
 s[1,1]  s[1,2]
 s[2,1]  s[2,2]
 s[3,1]  s[3,2]

In [5]:
@constraint(model, sum(s[1,j] for j=1:2) <= 200) # quantity supplied by s1

s[1,1] + s[1,2] <= 200

In [6]:
@constraint(model, sum(s[2,j] for j=1:2) <= 310) # quantity supplied by s2


s[2,1] + s[2,2] <= 310

In [7]:
@constraint(model, sum(s[3,j] for j=1:2) <= 420) # quantity supplied by s1

s[3,1] + s[3,2] <= 420

In [8]:
@constraint(model, sum(s[i,1] for i=1:3) <= 460) # plant A capacity

s[1,1] + s[2,1] + s[3,1] <= 460

In [9]:
@constraint(model, sum(s[i,2] for i=1:3) <= 560) # plant B capacity

s[1,2] + s[2,2] + s[3,2] <= 560

In [10]:
Supply_Rate=[1100,1000,900]

3-element Vector{Int64}:
 1100
 1000
  900

In [11]:
Supply_Cost=sum(Supply_Rate[i] * sum(s[i,j] for j=1:2) for i=1:3)

1100 s[1,1] + 1100 s[1,2] + 1000 s[2,1] + 1000 s[2,2] + 900 s[3,1] + 900 s[3,2]

In [12]:
Shipping_Rate=[[3000,3500],[2000,2500],[6000,4000]]

3-element Vector{Vector{Int64}}:
 [3000, 3500]
 [2000, 2500]
 [6000, 4000]

In [13]:
Shipping_Cost= sum(sum(Shipping_Rate[i][j] * s[i, j] for j = 1:2) for i = 1:3)

3000 s[1,1] + 3500 s[1,2] + 2000 s[2,1] + 2500 s[2,2] + 6000 s[3,1] + 4000 s[3,2]

In [14]:
Labour_Cost = 26000*sum(s[i,1] for i=1:3) + 21000 * sum(s[i,2] for i=1:3)

26000 s[1,1] + 26000 s[2,1] + 26000 s[3,1] + 21000 s[1,2] + 21000 s[2,2] + 21000 s[3,2]

In [15]:
Selling_Price = 50000 * sum(sum(s[i, j] for j = 1:2) for i = 1:3)

50000 s[1,1] + 50000 s[1,2] + 50000 s[2,1] + 50000 s[2,2] + 50000 s[3,1] + 50000 s[3,2]

In [16]:
Profit=Selling_Price-Shipping_Cost-Labour_Cost-Supply_Cost

19900 s[1,1] + 24400 s[1,2] + 21000 s[2,1] + 25500 s[2,2] + 17100 s[3,1] + 24100 s[3,2]

In [17]:
@objective(model,Max,Profit)

19900 s[1,1] + 24400 s[1,2] + 21000 s[2,1] + 25500 s[2,2] + 17100 s[3,1] + 24100 s[3,2]

In [18]:
@show model

model = A JuMP Model
Maximization problem with:
Variables: 6
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 5 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: CPLEX
Names registered in the model: s


A JuMP Model
Maximization problem with:
Variables: 6
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 5 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: CPLEX
Names registered in the model: s

In [19]:
optimize!(model)

CPLEX Error  3003: Not a mixed-integer problem.
Version identifier: 22.1.1.0 | 2022-11-26 | 9160aff4d
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual infeasibility =         73999.999998
Iteration:     3   Dual objective     =      23159000.000000


In [20]:
@show value.(s)

value.(s) = [200.0 0.0; 170.0 140.0; 0.0 420.0]


3×2 Matrix{Float64}:
 200.0    0.0
 170.0  140.0
   0.0  420.0

In [21]:
@show objective_value(model)

objective_value(model) = 2.1242e7


2.1242e7